This notebook demonstrates how to use OptKeras, wrapper of Keras and Optuna

##  Set up Google Colab environment

To run in Google Colab, specify a directory in Google Drive. (GPU is recommended.)

To run in an environment other than Google Colab, just skip this code.

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    # Specify a directory in Google Drive
    dir = '/content/drive/My Drive/Colab Notebooks/OptKeras'
    os.chdir(dir)    
    # Check the environment info
    print('## Current working directory: ', os.getcwd())
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
## Current working directory:  /content/drive/My Drive/Colab Notebooks/OptKeras
## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.0237722 hours (85.58 sec)
## Check the GPU info
Wed Feb 13 11:09:40 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+-----------------

## Install Optuna 0.7.0

In [2]:
!pip install optuna==0.7.0

    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 81kB 5.9MB/s 
    100% |████████████████████████████████| 51kB 21.3MB/s 
    100% |████████████████████████████████| 92kB 25.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/8c/97/c7/868e9cd5d6a23cfb98266f30d59131e5bd441132bdf8da18d5
  Stored in directory: /root/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built optuna pyperclip


## Install the latest version of OptKeras

In [3]:
!pip install git+https://github.com/Minyus/optkeras.git

  Cloning https://github.com/Minyus/optkeras.git to /tmp/pip-req-build-k8r4lchp
  Stored in directory: /tmp/pip-ephem-wheel-cache-rofjanl9/wheels/ad/c8/9d/67471f9fd7c5fd15961a2e7aa3d15215212d0631ea96df8b5c
Successfully built optkeras


## Import modules

In [4]:
# import Keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
import keras.backend as K

import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

import numpy as np
import pandas as pd

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

Using TensorFlow backend.


Keras 2.2.4
TensorFlow 1.13.0-rc1
Optuna 0.7.0
OptKeras 0.0.1


## Set up Dataset

In [5]:
dataset_name = 'MNIST'

if dataset_name in ['MNIST', 'MNIST_1000samples']:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)
    x_test = x_test.reshape(-1, img_x, img_y, 1)   
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    num_classes = 10
    input_shape = (img_x, img_y, 1)

11493376/11490434 [==============================] - 2s 0us/step


In [6]:
print('x_train: ', x_train.shape)
print('y_train', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test', y_test.shape)
print('input_shape: ', input_shape )    

x_train:  (60000, 28, 28, 1)
y_train (60000,)
x_test:  (10000, 28, 28, 1)
y_test (10000,)
input_shape:  (28, 28, 1)


## Try a simple Convolutional Neural Networks model

In [7]:
model = Sequential()
model.add(Conv2D(128, kernel_size = (3, 3), strides = (1, 1),
               activation = 'relu', input_shape = input_shape))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(optimizer = Adam(), 
            loss='sparse_categorical_crossentropy', metrics=['accuracy'])			  
model.fit(x_train, y_train, validation_data = (x_test, y_test), shuffle = True,
          batch_size = 512, epochs = 2) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 19s 325us/step - loss: 0.3435 - acc: 0.9036 - val_loss: 0.1476 - val_acc: 0.9576
Epoch 2/2
60000/60000 [==============================] - 6s 97us/step - loss: 0.1071 - acc: 0.9705 - val_loss: 0.0787 - val_acc: 0.9769


## Setup optimization of a Keras model

In [8]:
study_name = dataset_name + '_Optimized'

ok = OptKeras( 
    # parameters for optuna.create_study
    storage='sqlite:///' + study_name + '_Optuna.db', 
    sampler=None, 
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=0), 
    study_name = study_name, 
    direction='minimize', 
    load_if_exists = True,
    # parameters for OptKeras
    monitor='val_error', # Either 'val_error' (1 - val_acc) or 'val_loss'
    enable_pruning=False, 
    models_to_keep=1, # Either 1, 0, or -1 (save all models) 
    verbose=1 )

def objective(trial): 
    epochs = 10
    
    K.clear_session()   
    model = Sequential()
    
    if trial.suggest_int('Conv', 0, 1):  
        # 1 Convolution layer
        i = 1
        model.add(Conv2D(
            filters = int(trial.suggest_discrete_uniform(
                'Conv_{}_num_filters_'.format(i), 32, 64, 32)), 
            kernel_size=tuple([trial.suggest_int(
                'Conv_{}_kernel_size'.format(i), 2, 3)] * 2),
            activation='relu',
            input_shape = input_shape))
        model.add(MaxPooling2D(pool_size=tuple([trial.suggest_int(
                'Conv_{}_max_pooling_size'.format(i), 2, 3)] * 2)))
        model.add(Dropout(trial.suggest_discrete_uniform(
                'Conv_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        model.add(Flatten())        
    else:
        model.add(Flatten(input_shape=input_shape))
    # 2 Fully connected layers
    for i in np.arange(2) + 1:
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 256, 512, 256))))
        if trial.suggest_int('FC_{}_batch_normalization'.format(i), 0, 1):
            model.add(BatchNormalization())
        model.add(Activation(trial.suggest_categorical(
            'FC_{}_acivation'.format(i), ['relu'])))
        model.add(Dropout(
            trial.suggest_discrete_uniform(
                'FC_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        
    # Output layer    
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer_dict = { \
    #'Adagrad': Adagrad(),
    'Adam': Adam() }
    
    model.compile(optimizer = optimizer_dict[
        trial.suggest_categorical('Optimizer', list(optimizer_dict.keys()))],
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])    
    
    if ok.verbose >= 2: model.summary()
    
    batch_size = trial.suggest_int('Batch_size', 256, 256) 
    #batch_size = int(trial.suggest_discrete_uniform(
    #                  'Batch_size', 256, 512, 256) )
    data_augmentation = trial.suggest_int('Data_augmentation', 0, 1)
    
    if not data_augmentation:
        # [Required] Specify callbacks(trial) in fit method
        model.fit(x_train, y_train, batch_size = batch_size,
                  epochs = epochs, validation_data = (x_test, y_test),
                  shuffle = True,
                  callbacks = ok.callbacks(trial), 
                  verbose = ok.keras_verbose )
    
    if data_augmentation:
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            width_shift_range=[-1, 0, +1], # 1 pixel
            height_shift_range=[-1, 0, +1], # 1 pixel
            zoom_range=[0.95,1.05],  # set range for random zoom
            horizontal_flip=False,  # disable horizontal flip
            vertical_flip=False )  # disable vertical flip
        datagen.fit(x_train)
        # [Required] Specify callbacks(trial) in fit_generator method
        model.fit_generator(datagen.flow(x_train, y_train, 
                                         batch_size=batch_size),
                            epochs=epochs, validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // batch_size,
                            callbacks = ok.callbacks(trial), 
                            verbose = ok.keras_verbose )  
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

[I 2019-02-13 11:10:45,703] A new study created with name: MNIST_Optimized


[2019-02-13 11:10:45.727697]  Ready for optimization. (message printed as verbose is set to 1+)


## Run optimization trials

In [9]:
# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, n_trials=10, timeout = 12 * 60 * 60)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-02-13 11:10:46.894130] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None


[I 2019-02-13 11:13:27,198] Finished a trial resulted in value: 0.014599999999999946. Current best value is 0.014599999999999946 with parameters: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.0, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}.


[2019-02-13 11:13:28.263281] Latest trial id: 1, value: 0.014599999999999946 (TrialState.COMPLETE) | Best trial id: 1, value: 0.014599999999999946, parameters: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.0, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}


[I 2019-02-13 11:16:19,177] Finished a trial resulted in value: 0.010800000000000032. Current best value is 0.010800000000000032 with parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.5, 'Conv_1_kernel_size': 2, 'Conv_1_max_pooling_size': 3, 'Conv_1_num_filters_': 32.0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}.


[2019-02-13 11:16:20.169901] Latest trial id: 2, value: 0.010800000000000032 (TrialState.COMPLETE) | Best trial id: 2, value: 0.010800000000000032, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.5, 'Conv_1_kernel_size': 2, 'Conv_1_max_pooling_size': 3, 'Conv_1_num_filters_': 32.0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.5, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.25, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}


...

[2019-02-13 11:26:53.181970] Latest trial id: 8, value: 0.009199999999999986 (TrialState.COMPLETE) | Best trial id: 3, value: 0.009000000000000008, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 2, 'Conv_1_num_filters_': 32.0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.25, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.5, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}


[I 2019-02-13 11:29:31,779] Finished a trial resulted in value: 0.013700000000000045. Current best value is 0.009000000000000008 with parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 2, 'Conv_1_num_filters_': 32.0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.25, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.5, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}.


[2019-02-13 11:29:32.746838] Latest trial id: 9, value: 0.013700000000000045 (TrialState.COMPLETE) | Best trial id: 3, value: 0.009000000000000008, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 2, 'Conv_1_num_filters_': 32.0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.25, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.5, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}


[I 2019-02-13 11:29:54,871] Finished a trial resulted in value: 0.01649999999999996. Current best value is 0.009000000000000008 with parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 2, 'Conv_1_num_filters_': 32.0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.25, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.5, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}.


[2019-02-13 11:29:54.971461] Latest trial id: 10, value: 0.01649999999999996 (TrialState.COMPLETE) | Best trial id: 3, value: 0.009000000000000008, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 2, 'Conv_1_num_filters_': 32.0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.25, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.5, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}


## Check the results

In [10]:
# OptKeras best_trial returns 
print('Best trial id: ', ok.best_trial.trial_id)
print('Best value:', ok.best_trial.value)
print('Best params: ')
ok.best_trial.params

Best trial id:  3
Best value: 0.009000000000000008
Best params: 


{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.25,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 2,
 'Conv_1_num_filters_': 32.0,
 'Data_augmentation': 1,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 1,
 'FC_1_dropout_rate': 0.25,
 'FC_1_num_hidden_units': 512.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 0,
 'FC_2_dropout_rate': 0.5,
 'FC_2_num_hidden_units': 512.0,
 'Optimizer': 'Adam'}

In [11]:
"""
Alternatively, you can access Optuna's study object to, for example, 
get the best parameters as well.
Please note that study.best_trial returns error if optimization trials 
were not completed (e.g. if you interupt execution) as of Optuna 0.7.0, 
so usage of OptKeras is recommended.
"""
study = ok.study
study.best_trial.params 

{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.25,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 2,
 'Conv_1_num_filters_': 32.0,
 'Data_augmentation': 1,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 1,
 'FC_1_dropout_rate': 0.25,
 'FC_1_num_hidden_units': 512.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 0,
 'FC_2_dropout_rate': 0.5,
 'FC_2_num_hidden_units': 512.0,
 'Optimizer': 'Adam'}

In [12]:
## Check the Optuna CSV log file 
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
pd.read_csv(ok.optuna_log_file_path)

Data Frame read from MNIST_Optimized_Optuna.csv 



,trial_id,state,value,datetime_start,datetime_complete,Batch_size,Conv,Conv_1_dropout_rate,Conv_1_kernel_size,Conv_1_max_pooling_size,...,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,1,TrialState.COMPLETE,0.0146,2019-02-13 11:10:45.749164,2019-02-13 11:13:27.165666,256,0,NaN,NaN,NaN,...,2019-02-13 11:13:11.655200,2019-02-13 11:13:26.875603,val_error,1,0.971093,0.028907,0.091096,0.9854,0.0146,0.042454
1,2,TrialState.COMPLETE,0.0108,2019-02-13 11:13:27.207080,2019-02-13 11:16:19.141811,256,1,0.50,2.0,3.0,...,2019-02-13 11:16:02.671262,2019-02-13 11:16:18.805748,val_error,2,0.968583,0.031417,0.098159,0.9892,0.0108,0.030241
2,3,TrialState.COMPLETE,0.0090,2019-02-13 11:16:19.182370,2019-02-13 11:19:09.019019,256,1,0.25,3.0,2.0,...,2019-02-13 11:18:52.768998,2019-02-13 11:19:08.668811,val_error,3,0.986442,0.013558,0.044292,0.9910,0.0090,0.029543
3,4,TrialState.COMPLETE,0.0115,2019-02-13 11:19:09.065058,2019-02-13 11:21:56.108416,256,1,0.25,2.0,3.0,...,2019-02-13 11:21:39.839719,2019-02-13 11:21:55.711689,val_error,4,0.986492,0.013508,0.042585,0.9885,0.0115,0.040009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,7,TrialState.COMPLETE,0.0198,2019-02-13 11:23:34.600983,2019-02-13 11:23:56.843933,256,0,NaN,NaN,NaN,...,2019-02-13 11:23:54.765027,2019-02-13 11:23:56.544645,val_error,7,0.995017,0.004983,0.014111,0.9802,0.0198,0.073235
7,8,TrialState.COMPLETE,0.0092,2019-02-13 11:23:56.901083,2019-02-13 11:26:51.845384,256,1,0.25,3.0,2.0,...,2019-02-13 11:26:35.143727,2019-02-13 11:26:51.355880,val_error,8,0.993054,0.006946,0.020988,0.9908,0.0092,0.031861
8,9,TrialState.COMPLETE,0.0137,2019-02-13 11:26:51.906647,2019-02-13 11:29:31.722901,256,0,NaN,NaN,NaN,...,2019-02-13 11:29:15.865533,2019-02-13 11:29:30.733310,val_error,9,0.979362,0.020638,0.065974,0.9863,0.0137,0.041371
9,10,TrialState.COMPLETE,0.0165,2019-02-13 11:29:31.792148,2019-02-13 11:29:54.812921,256,0,NaN,NaN,NaN,...,2019-02-13 11:29:52.681523,2019-02-13 11:29:54.419806,val_error,10,0.992100,0.007900,0.023792,0.9835,0.0165,0.052651


In [13]:
## Check the Keras CSV log file

pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
pd.read_csv(ok.keras_log_file_path)

Data Frame read from MNIST_Optimized_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,0,2019-02-13 11:10:47.846167,2019-02-13 11:11:03.975231,val_error,1,0.874048,0.125952,0.403773,0.9608,0.0392,0.120862
1,1,2019-02-13 11:11:04.551341,2019-02-13 11:11:19.585107,val_error,1,0.937182,0.062818,0.203350,0.9760,0.0240,0.077140
2,2,2019-02-13 11:11:20.358317,2019-02-13 11:11:35.590956,val_error,1,0.948597,0.051403,0.163696,0.9763,0.0237,0.072022
3,3,2019-02-13 11:11:36.467452,2019-02-13 11:11:51.702920,val_error,1,0.955611,0.044389,0.140955,0.9795,0.0205,0.060727
...,...,...,...,...,...,...,...,...,...,...,...
96,6,2019-02-13 11:29:46.625672,2019-02-13 11:29:48.372567,val_error,10,0.990550,0.009450,0.030616,0.9811,0.0189,0.058786
97,7,2019-02-13 11:29:48.647015,2019-02-13 11:29:50.383037,val_error,10,0.990217,0.009783,0.028173,0.9810,0.0190,0.062615
98,8,2019-02-13 11:29:50.667720,2019-02-13 11:29:52.401896,val_error,10,0.991450,0.008550,0.026595,0.9811,0.0189,0.062924
99,9,2019-02-13 11:29:52.681523,2019-02-13 11:29:54.419806,val_error,10,0.992100,0.007900,0.023792,0.9835,0.0165,0.052651
